In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
from astropy.table import Table
import math
import Corrfunc
from os.path import dirname, abspath, join as pjoin
from Corrfunc.io import read_catalog
from copy import deepcopy
import time

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
from dataloc import *
import plotting as pp
import groupcatalog as gc
from nnanalysis import *
import catalog_definitions as cat
from redshift_guesser import SimpleRedshiftGuesser, PhotometricRedshiftGuesser
from groupcatalog import TestGroupCatalog, BGSGroupCatalog
from calibrationdata import CalibrationData
import hod
import bgs_helpers

%load_ext autoreload
%autoreload 2

# timing helper
def time_func(func, args, repeats=5):
    t0 = time.perf_counter()
    out = None
    for _ in range(repeats):
        out = func(*args)
    t1 = time.perf_counter()
    return (t1 - t0) / repeats, out

# Unit tests

In [ ]:
# Test that identifying unobserved galaxies from a table works correctly in all cases

# Create a test astropy table
table = Table()
# Add a masked column named 'Z'
table['Z'] = np.ma.array([0.1, np.nan, 99.9, 0.4, 0.5], mask=[False, False, False, True, False])
unobserved_truth = np.array([False, True, True, True, False])
test = bgs_helpers.determine_unobserved_from_z(table['Z'])

assert (test == unobserved_truth).all(), f"determine_unobserved_from_z did not work as expected: {test}"


In [ ]:
ld_cache = LuminosityDistanceCache()

# Generate a large array of random redshifts
test_z = np.random.uniform(0.001, 0.5, 100000)

# --- Performance Test ---

# 1. Using the cache
start_time = time.perf_counter()
dist_cached = ld_cache.luminosity_distance(test_z)
end_time = time.perf_counter()
print(f"Cached lookup took: {end_time - start_time:.6f} seconds")

# 2. Using astropy directly (will be much slower)
start_time = time.perf_counter()
dist_astropy = get_cosmology().luminosity_distance(test_z).to(u.Mpc).value
end_time = time.perf_counter()
print(f"Astropy direct call took: {end_time - start_time:.6f} seconds")

# --- Verification ---

# Check that the interpolated values are very close to the direct calculation
max_rel_error = np.max(np.abs(dist_cached - dist_astropy) / dist_astropy)
print(f"Maximum relative error: {max_rel_error:.2e}")
assert max_rel_error < 1e-6 # Should be very accurate with enough points

# --- Roundtrip Test ---
test_dist = np.random.uniform(10, 2000, 1000)
z_from_dist = ld_cache.z_at_value(test_dist)
dist_roundtrip = ld_cache.luminosity_distance(z_from_dist)
assert np.allclose(test_dist, dist_roundtrip, rtol=1e-4)

In [ ]:
logL = 10
answer1 = SOLAR_L_R_BAND - 2.5 * logL
answer2 =  SOLAR_L_R_BAND - (logL / 0.39794) # No idea why I used to do this
print(f"answer1: {answer1}, answer2: {answer2}")

In [ ]:
mag = np.random.uniform(-25, -11, 1000)
logL = abs_mag_r_to_log_solar_L(mag)
mag_after = log_solar_L_to_abs_mag_r(logL)
assert np.allclose(mag, mag_after), "Magnitude conversion didn't round trip correctly"

lum = abs_mag_r_to_solar_L(mag)
mag_after = log_solar_L_to_abs_mag_r(np.log10(lum))
assert np.allclose(mag, mag_after), "Magnitude conversion didn't round trip correctly"

In [ ]:
import time
import numpy as np

# prepare test redshifts
z = np.random.uniform(0.001, 0.5, 10000)

# warm up / correctness check
astropy_volumes = get_cosmology().comoving_volume(z).value
my_volumes = get_volume_at_z(z, 1.0)
assert np.allclose(astropy_volumes, my_volumes), f"Volume mismatch (max abs diff={np.max(np.abs(astropy_volumes-my_volumes)):.3e})"

# timing helper
def time_func(func, args, repeats):
    t0 = time.perf_counter()
    out = None
    for _ in range(repeats):
        out = func(*args)
    t1 = time.perf_counter()
    return (t1 - t0) / repeats, out

repeats = 10
t_ast, out_ast = time_func(lambda zz: get_cosmology().comoving_volume(zz).value, (z,), repeats=repeats)
t_my, out_my   = time_func(lambda zz: get_volume_at_z(zz, 1.0), (z,), repeats=repeats)

print(f"Astropy comoving_volume: {t_ast:.6f} s/call (avg over {repeats})")
print(f"get_volume_at_z         : {t_my:.6f} s/call (avg over {repeats})")
print(f"Astropy / mine ratio    : {t_ast / t_my:.3f}")

# numerical comparison summary
max_abs = np.max(np.abs(out_ast - out_my))
max_rel = max_abs / np.max(np.abs(out_ast))
print(f"Max abs diff = {max_abs:.6e}, Max rel diff = {max_rel:.3e}")

In [ ]:
mags = np.array([-19.5, -20.5, -21.5])
vmax = get_max_observable_volume(mags, 0.01, 0.5, 19.5, 1.0)  
assert vmax[0] < vmax[1] < vmax[2], "Expected vmax to increase with increasing luminosity"

vmax2 = get_max_observable_volume(mags, 0.1, 0.5, 19.5, 1.0)  
assert vmax2[0] < vmax[0], "Expected vmax to decrease with increasing z_min"

vmax3 = get_max_observable_volume(mags, 0.01, 0.2, 19.5, 1.0)
assert vmax3[2] < vmax[2], "Expected vmax to decrease with decreasing z_max if the galaxy is visible at z_max"

vmax4 = get_max_observable_volume(mags, 0.01, 0.4, 19.5, 1.0)
assert vmax4[0] == vmax[0], "Expected vmax to stay the same if the galaxy is not visible at z_max"

In [ ]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_dn4000(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# Basic test of multiple versions of SimpleRedshiftGuesser
# Ensure it handles arrays of inputs and gives a reasonable answer for a couple obvious cases

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])

# Neighbor properties
nn_z = np.array([0.1, 0.2, 0.3])
nn_dist = np.array([250.0, 3.0, 30.0])
nn_q = np.array([True, True, False])

simple = SimpleRedshiftGuesser(None, None, ver='5.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='4.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.12, z[2]

simple = SimpleRedshiftGuesser(None, None, ver='2.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1


In [ ]:
# Basic test on scores from the SV3 bins file 
target_neighbor= [1,1,1,1,1,8]
target_app_mag = [19.0, 19.0, 18.0, 14.0, 19.0, 19.0]
target_quiescent = [1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
neighbor_z = [0.1, 0.2, 0.05, 0.3, 0.1, 0.1]
neighbor_ang_dist = [30.0, 150.0, 2.0, 15.0, 60.0, 60.0]
nn_quiescent = [1.0, 0.0, 1.0, 1.0, 1.0, 1.0]

nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE_V2)
#nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE)
score_b = nna.get_score(target_neighbor, target_app_mag, target_quiescent, neighbor_z, neighbor_ang_dist, nn_quiescent)
print(score_b)

assert score_b[0] > 0.01, f"Reasonable parameters should have a non-zero score, but got {score_b[0]}"
#assert score_b[1] < 0.1, f"Very high angular distance should have a low score, but got {score_b[1]}" # BUG is it that issue I saw where edge value is True and spreads?
assert score_b[2] > 0.4, f"Very low angular distance should have a high score, but got {score_b[2]}"
assert score_b[3] < 0.1, f"Bright target with neighbor at high z should have a low score even at close distance {score_b[3]}"


In [ ]:
# Test that Scipy linear interp is doing what we want inside the NNAnalyzer
nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE_V2)
frac, all_counts, simz_counts = nna.integrate_out_dimension((0,6))

print(frac.shape)
first=frac[0,0,5,9,5]
second=frac[0,0,5,10,5]
mid = (first+second)/2
print(first, second, mid)

score = nna.get_score(None, [(APP_MAG_BINS[9]+APP_MAG_BINS[10]) / 2], [0.0], [Z_BINS[5]], [ANGULAR_BINS[5]], [0.0])

assert np.isclose(mid, score)

In [ ]:
# Basic tests for PhotometricRedshiftGuesser

# Target (lost galaxies) properties
t_app_mag = np.array([19.0, 18.0, 12.0, 17.1])
t_pobs = np.array([0.5, 0.5, 0.5, 0.9])
t_q = np.array([True, True, False, False])
t_zphot = np.array([0.105, 0.230, 0.011, 0.070])

# Neighbor properties
nn_z = np.array([[0.1, 0.2, 0.3, 0.35],
                 [0.2, 0.3, 0.1, 0.03]])
nn_dist = np.array([[250.0, 3.0,  30.0, 4.0],
                    [260.0, 40.0, 40.0, 8.0]])
nn_q = np.array([[True, True, False, True],
                 [False, False, False, True]])

scorer = PhotometricRedshiftGuesser.from_files(BGS_Y3_LOST_APP_TO_Z_FILE, BGS_Y3_LOST_APP_AND_ZPHOT_TO_Z_FILE, NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE_V2, Mode.PHOTOZ_PLUS_v3)
scorer.debug = True
params = ([0.8104, 0.9215, 2.867 ], [0.9102, 0.7376, 3.0275], [0.8986, 1.0397, 2.6287], [0.7488, 0.9489, 2.9319]) # 3. params

z, assignment_type = scorer.choose_redshift(nn_z, nn_dist, t_zphot, t_pobs, t_app_mag, t_q, nn_q, params)

print(z)
print(assignment_type)

# TODO more tests

assert assignment_type[0] <= -1, "Should not use a neighbor"
assert assignment_type[1] == 1, "should use neighbor 1 "
assert assignment_type[2] <= -1, "should not use a neighbor"
assert assignment_type[3] == 2, "should use neighbor 2"

assert np.isnan(z).sum() == 0, "z should not have any NaNs"
assert z[0] > 0.0, "z[0] should be greater than 0.0"
assert z[1] == 0.2, "z[1] should be 0.2"
assert z[2] < 0.1, "z[2] should be less than 0.1"
assert z[3] == 0.03, "z[3] should be 0.03"

In [ ]:
# Test out sim_z_score function
# Test out smooth redshift comparison function works as desired for the relevant redshift differences
x=np.arange(-0.100, 0.105, 0.0005)
plt.plot(x, powerlaw_score_1(0.2, 0.2+x), label="Powerlaw 1")
plt.axvline(0.005, color='r')
plt.axvline(-0.005, color='r')
plt.plot(x, powerlaw_score_2(0.2, 0.2+x), color='purple', label="Powerlaw 2")
plt.plot(x, rounded_tophat_score(0.2, 0.2-x), color='g', label="close enough smooth")
plt.tight_layout()
plt.legend()
plt.xlabel("Redshift difference")
plt.ylabel("Similarity score")

# Make sure the extremes are working as expected
assert powerlaw_score_1(0.1, 0.3) < 0.01
assert powerlaw_score_1(0.2, 0.3) < 0.05 and powerlaw_score_1(0.2, 0.3) > 0.01
assert powerlaw_score_1(0.2, 0.25) < 0.1 and powerlaw_score_1(0.2, 0.25) > 0.05
assert powerlaw_score_1(0.2, 0.210) > 0.2, powerlaw_score_1(0.2, 0.210)
assert powerlaw_score_1(0.2, 0.205) > 0.95, powerlaw_score_1(0.2, 0.205)
assert powerlaw_score_1(0.2, 0.203) > 0.99, powerlaw_score_1(0.2, 0.203)
assert np.isclose(powerlaw_score_1(0.2, 0.2001), 1.0)


### CIC Binning

In [ ]:
# 1D test of my N-dimensional CIC binning function
data_1d = np.array([
    0.0, 
    3.5, 
])
first_dim  = np.linspace(0, 5, 6)

bin_counts = cic_binning(data_1d, [first_dim])
print(bin_counts)

assert np.shape(bin_counts) == (6,), np.shape(bin_counts)
assert np.sum(bin_counts) == len(data_1d), np.sum(bin_counts)
assert np.isclose(bin_counts[0], 1)
assert np.isclose(bin_counts[3], 0.5)
assert np.isclose(bin_counts[4], 0.5)
assert np.isclose(bin_counts[5], 0.0)

In [ ]:
# 2D test of my N-dimensional CIC binning function
data_2d = np.array([
    [0.0, 0.0], # Test corner case
    [0.0, -0.5], # Test left edge case
    [3.5, 3.5], # Test middle case
    [2, 5.9],
    [0.5, 40.0], # Test right edge case
    [-7.0, -3.0], # Extra edge case
])
first_dim  = np.linspace(0, 5, 6)
second_dim  = np.linspace(0, 6, 7)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (6, 7), np.shape(bin_counts)
assert np.sum(bin_counts) == len(data_2d), np.sum(bin_counts)
assert np.isclose(bin_counts[0,0], 3.0), bin_counts[0,0]
assert np.isclose(bin_counts[2,5], 0.1), bin_counts[2,5]
assert np.isclose(bin_counts[2,6], 0.9), bin_counts[2,6]
assert np.isclose(bin_counts[3,3], 0.25), bin_counts[3,3]
assert np.isclose(bin_counts[3,4], 0.25), bin_counts[3,4]
assert np.isclose(bin_counts[4,3], 0.25), bin_counts[4,3]
assert np.isclose(bin_counts[4,4], 0.25), bin_counts[4,4]
assert np.isclose(bin_counts[4,4], 0.25), bin_counts[4,4]
assert np.isclose(bin_counts[0,6], 0.5), bin_counts[0,6]
assert np.isclose(bin_counts[1,6], 0.5), bin_counts[1,]

In [ ]:
# 3D test of CIC binning
data_3d = np.array([
    [0.0, 0.0, 0.0],  # Test corner case
    [1.5, 1.5, 1.5],  # Test middle case
    [10.0, 10.0, -10.0],  #  edge case
    [0.0, -1.0, 1.6]
])
first_dim_3d = np.linspace(0, 3, 4)
second_dim_3d = np.linspace(0, 4, 5)
third_dim_3d = np.linspace(0, 2, 3)

# Perform CIC binning
bin_counts_3d = cic_binning(data_3d, [first_dim_3d, second_dim_3d, third_dim_3d])
print(bin_counts_3d)

# Assertions to verify the binning results
assert np.shape(bin_counts_3d) == (4, 5, 3), np.shape(bin_counts_3d)
assert np.isclose(np.sum(bin_counts_3d), len(data_3d)), np.sum(bin_counts_3d)
assert np.isclose(bin_counts_3d[0,0,0], 1.0), bin_counts_3d[0,0,0]
assert np.isclose(bin_counts_3d[1,1,1], 1/8), bin_counts_3d[1,1,1]
assert np.isclose(bin_counts_3d[1,1,2], 1/8), bin_counts_3d[1,1,2]
assert np.isclose(bin_counts_3d[1,2,1], 1/8), bin_counts_3d[1,2,1]
assert np.isclose(bin_counts_3d[1,2,2], 1/8), bin_counts_3d[1,2,2]
assert np.isclose(bin_counts_3d[2,1,1], 1/8), bin_counts_3d[2,1,1]
assert np.isclose(bin_counts_3d[2,1,2], 1/8), bin_counts_3d[2,1,2]
assert np.isclose(bin_counts_3d[2,2,1], 1/8), bin_counts_3d[2,2,1]
assert np.isclose(bin_counts_3d[2,2,2], 1/8), bin_counts_3d[2,2,2]
assert np.isclose(bin_counts_3d[3,4,0], 1.0), bin_counts_3d[3,4,0]
assert np.isclose(bin_counts_3d[0,0,1], 0.4), bin_counts_3d[3,4,1]
assert np.isclose(bin_counts_3d[0,0,2], 0.6), bin_counts_3d[3,4,1]

In [ ]:
# Test CIC binning with weights
data_2d = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
])
first_dim  = np.linspace(0, 1, 2)
second_dim  = np.linspace(0, 1, 2)

bin_counts = cic_binning(data_2d, [first_dim, second_dim], weights=[0.66, 2.99])
print(bin_counts)

assert np.shape(bin_counts) == (2, 2), np.shape(bin_counts)
assert np.isclose(bin_counts[0,0], 0.66), bin_counts[0,0]
assert np.isclose(bin_counts[0,1], 2.99), bin_counts[0,0]

In [ ]:
# Test CIC binning with repeats
data_2d = np.array([
    [0.0, 0.0],  
    [1.0, 1.0],  
    [1.0, 1.0],  
    [1.0, 1.0],  
])
first_dim  = np.linspace(0, 1, 2)
second_dim  = np.linspace(0, 1, 2)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (2, 2), np.shape(bin_counts)
assert np.isclose(bin_counts[0,0], 1.0), bin_counts[0,0]
assert np.isclose(bin_counts[0,1], 0.0), bin_counts[0,1]
assert np.isclose(bin_counts[1,0], 0.0), bin_counts[1,0]
assert np.isclose(bin_counts[1,1], 3.0), bin_counts[1,1]

In [ ]:
# Test CIC binning with negative dimensional ranges
data_2d = np.array([
    [-1.0, -1.2],  
    [0.5, 1.0],  
])
first_dim  = np.linspace(-1, 1, 3)
second_dim  = np.linspace(-1, 1, 3)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (3, 3), np.shape(bin_counts)
assert np.isclose(np.sum(bin_counts), 2.0), bin_counts[0,0]
assert np.isclose(bin_counts[0,0], 1.0), bin_counts[0,0]
assert np.isclose(bin_counts[1,2], 0.5), bin_counts[1,1]
assert np.isclose(bin_counts[2,2], 0.5), bin_counts[1,1]

In [ ]:
# Test CIC binning with log data
data_2d = np.array([
    [3.0, 1.0],  # 1000.0, 1.0
    [1.5, 2.0],  # 31.6, 2.0
])
first_dim  = np.linspace(1, 4, 4) # log data though; 10 100 1000 10000
second_dim  = np.linspace(0, 3, 4)

bin_counts = cic_binning(data_2d, [first_dim, second_dim], logscale=[10, False])
print(bin_counts)
print(data_2d)

assert np.shape(bin_counts) == (4, 4), np.shape(bin_counts)
assert np.isclose(np.sum(bin_counts), 2.0), np.sum(bin_counts)
assert np.isclose(bin_counts[2,1], 1.0), bin_counts[2,1]
assert bin_counts[0,2] > 0.75, bin_counts[0,2] # 10^1.5 ~ 31.6, compared to 10 vs 100
assert bin_counts[1,2] < 0.25, bin_counts[0,2]

In [ ]:
# Test CIC binning with log data
data = np.array([16.4, 18.05])
print(APP_MAG_BINS)

bin_counts = cic_binning(data, [APP_MAG_BINS], logscale=[2.5])
print(bin_counts)

assert np.shape(bin_counts) == np.shape(APP_MAG_BINS)
assert bin_counts[2] > 0.7, bin_counts[2]
assert bin_counts[5] < 0.1, bin_counts[2]
assert bin_counts[6] > 0.9, bin_counts[2]
assert bin_counts[7] == 0, bin_counts[2]


In [ ]:
# Stress test of CIC binning
data_stress = np.random.rand(1000000, 5) # 5M rows of random data
dim_stress = np.linspace(0, 1, 11)

# Perform CIC binning
bin_counts_stress = cic_binning(data_stress, [dim_stress, dim_stress, dim_stress, dim_stress, dim_stress])   

# Assertions to verify the binning results
assert np.shape(bin_counts_stress) == (11, 11, 11, 11, 11), np.shape(bin_counts_stress)
assert np.isclose(np.sum(bin_counts_stress), len(data_stress)), np.sum(bin_counts_stress)

with np.printoptions(precision=1, suppress=True, linewidth=100):
    print(np.sum(np.sum(np.sum(bin_counts_stress, axis=0), axis=0), axis=0))

In [ ]:
# Test CIC binning with negative dimensional ranges
data_2d = np.array([
    [-0.01, APP_MAG_BINS[3]],  
    [0.01, APP_MAG_BINS[3]],  
    [0.03, APP_MAG_BINS[3]],  
    [0.12, APP_MAG_BINS[3]],  
    [0.21, APP_MAG_BINS[3]],  
    [0.40, APP_MAG_BINS[3]],  
    [0.50, APP_MAG_BINS[5]],  
])
first_dim  = Z_BINS
second_dim  = APP_MAG_BINS

bin_counts = cic_binning(data_2d, [first_dim, second_dim])

with np.printoptions(precision=3, suppress=True, linewidth=200):
    print(bin_counts)



In [ ]:
# 1D test with uneven bins
data_1d = np.array([
    2.5,
    4.0, 
    14.0,
])
first_dim  = np.array([0, 1, 3, 4, 5, 15])

bin_counts = cic_binning(data_1d, [first_dim])
print(bin_counts)

assert np.shape(bin_counts) == (6,), np.shape(bin_counts)
assert np.sum(bin_counts) == len(data_1d), np.sum(bin_counts)
assert np.isclose(bin_counts[0], 0)
assert np.isclose(bin_counts[1], 0.25)
assert np.isclose(bin_counts[2], 0.75)
assert np.isclose(bin_counts[3], 1.0)
assert np.isclose(bin_counts[4], 0.1)
assert np.isclose(bin_counts[5], 0.9)

### Other

In [ ]:
# Question: which is faster, boolean indexing or integer indexing?
arr = np.random.rand(20000000)
bool_filter = arr < 0.1
N = 20

test = np.zeros(bool_filter.sum())
test2 = np.zeros(len(arr))
t1 = time.time()
for n in range(N):
    test += arr[bool_filter]
    test2[bool_filter] += arr[bool_filter]
t2 = time.time()
print(f"Time for bool filter: {t2-t1}")

test = np.zeros(bool_filter.sum())
t1 = time.time()
idx_filter = np.flatnonzero(bool_filter)
for n in range(N):
    test += arr[idx_filter]
    test2[idx_filter] += arr[idx_filter]
t2 = time.time()
print(f"Time for idx filter: {t2-t1}")

# Answer: idx filter is faster. And order doesn't matter

In [ ]:
import time

# function signature is write_dat_files(ra, dec, z_eff, log_L_gal, V_max, colors, chi, outname_base, frac_area):

# Generate some sample data to write
NUM_ROWS = 500000
ra = np.random.rand(NUM_ROWS)
dec = np.random.rand(NUM_ROWS)
z_eff = np.random.rand(NUM_ROWS)
log_L_gal = np.random.rand(NUM_ROWS)
V_max = np.random.rand(NUM_ROWS)
colors = np.random.randint(0, 2, NUM_ROWS)
chi = np.random.rand(NUM_ROWS)
outname_base2 = OUTPUT_FOLDER + 'speed-write-test2'

# Write data using write_dat_files
start_time = time.time()
write_dat_files(ra, dec, z_eff, log_L_gal, V_max, colors, chi, outname_base2)
end_time = time.time()
time_v2 = end_time - start_time

# Compare the contents of the two files
with open(outname_base2 + '.dat', 'rb') as f2:
    data_v2 = f2.read()

#assert data_v1.strip() == data_v2.strip(), "The outputs of write_dat_files and write_dat_files_v2 are not the same"

# Print the time taken by each function
print(f"Time taken by write_dat_files_v2: {time_v2} seconds")

In [ ]:
dataMat1 = np.random.rand(1000,1000)
dataMat2 = np.random.rand(2,500000)
dataMat3 = np.random.rand(500000,2)
start = time.perf_counter()
with open(OUTPUT_FOLDER + 'test1.txt','w') as f:
    np.savetxt(f,dataMat1,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER +  'test2.txt','w') as f:
    np.savetxt(f,dataMat2,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER +  'test3.txt','w') as f:
    np.savetxt(f,dataMat3,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER + 'test4.txt','w') as f:
    fmt = ' '.join(['%g']*dataMat3.shape[1])
    fmt = '\n'.join([fmt]*dataMat3.shape[0])
    data = fmt % tuple(dataMat3.ravel())        
    f.write(data)
end = time.perf_counter()
print(end-start)

# End to End Test

In [ ]:
# Only needs to be run once, unless you want to change the test data
#catalog = TestGroupCatalog("Test")
#catalog.create_test_dat_files() 

In [ ]:
testcat = deepcopy(cat.bgs_sv3_pz_2_6_10p_c2)
testcat.sdss_fill = False
testcat.preprocess()

# read in and print out the first few lines of SV3_test.preprocess_file
with open(testcat.preprocess_file, 'r') as f:
    for i in range(10):
        print(f.readline(), end='')

galprops_file = str.replace(testcat.GF_outfile, ".out", "_galprops.pkl")
galprops = pickle.load(open(galprops_file, "rb"))
print(galprops[0:10])

In [ ]:
# Test an orphaned satellite in final iteration scenario.
catalog = TestGroupCatalog("Test")
catalog.GF_props['iterations'] = 1 # By running with 1 iteration only on these data, we expose a situation where satellites are orphaned in the final iteration
catalog.run_group_finder(silent=True) 
catalog.postprocess()
#df=catalog.all_data.loc[catalog.all_data['Z'] < 0.1]
#pp.examine_area(np.min(df.RA), np.max(df.RA), np.min(df['DEC']), np.max(df['DEC']), df)
catalog.sanity_tests(skiphod=True) # Includes tests on the orphaned satellites, effectively.


In [ ]:
catalog = TestGroupCatalog("Test")
catalog.GF_props['iterations'] = 5 
catalog.run_group_finder(silent=True) 
catalog.postprocess()
#df=catalog.all_data.loc[catalog.all_data['Z'] < 0.1]
#pp.examine_area(np.min(df.RA), np.max(df.RA), np.min(df['DEC']), np.max(df['DEC']), df)
catalog.sanity_tests(skiphod=True) # Includes tests on the orphaned satellites, effectively.

In [ ]:
# C Group Finder Tests

np.set_printoptions(threshold=sys.maxsize)

# Baseline vanilla group finder test 
catalog = TestGroupCatalog("Test")
catalog.run_group_finder(silent=True) 
catalog.postprocess()
df=catalog.all_data
baseline_total_mass = df['M_HALO'].sum()
assert len(np.unique(df['IGRP'])) == 200, len(np.unique(df['IGRP']))
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129

assert np.isclose(df['WEIGHT'].sum(), 246 * 1.0) # no weights, just 1 per gal
m1=df['M_HALO'].to_numpy()

# Test that when omega0 are 0, the others don't matter
catalog = TestGroupCatalog("Test")
catalog.GF_props['omegaL_sf'] = 123
catalog.GF_props['sigma_sf'] = 345
catalog.GF_props['omegaL_q'] = 456
catalog.GF_props['sigma_q'] = 678
catalog.GF_props['omega0_sf'] = 0.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['IGRP'])) == 200, len(np.unique(df['IGRP']))
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129
assert np.isclose(df['WEIGHT'].sum(), 246 * 1.0) # no weights, just 1 per gal
assert np.isclose(df['M_HALO'].sum(), baseline_total_mass)
m2=df['M_HALO'].to_numpy()

catalog = TestGroupCatalog("Test")
#catalog.GF_props['colors'] = 1
catalog.GF_props['omegaL_sf'] = 10.0
catalog.GF_props['sigma_sf'] = 3.0
catalog.GF_props['omegaL_q'] = 0.0
catalog.GF_props['sigma_q'] = 0.0
catalog.GF_props['omega0_sf'] = 10.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['IGRP'])) >= 200 # these parameters make assigned halos smaller
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129
assert df['WEIGHT'].sum() < 246 
# When you change parameters, we get a different number of gropus and thus total mass.
# assert np.isclose(df['M_HALO'].sum(), baseline_total_mass), f"Expected {np.log10(baseline_total_mass)}, got {np.log10(df['M_HALO'].sum())}"
m3=df['M_HALO'].to_numpy()

plt.hist(np.stack([np.log10(m1), np.log10(m2), np.log10(m3)], axis=-1))



print("All tests passed")

In [ ]:
# C Group Finder Tests

np.set_printoptions(threshold=sys.maxsize)

# Baseline vanilla group finder test 
catalog = TestGroupCatalog("Test")
catalog.run_group_finder(silent=True) 
catalog.postprocess()
df=catalog.all_data
baseline_total_mass = df['M_HALO'].sum()
# TODO this result is not stable...
assert len(np.unique(df['IGRP'])) == 200, len(np.unique(df['IGRP']))
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129

assert np.isclose(df['WEIGHT'].sum(), 246 * 1.0) # no weights, just 1 per gal
m1=df['M_HALO'].to_numpy()

# Test that when omega0 are 0, the others don't matter
catalog = TestGroupCatalog("Test")
catalog.GF_props['omegaL_sf'] = 123
catalog.GF_props['sigma_sf'] = 345
catalog.GF_props['omegaL_q'] = 456
catalog.GF_props['sigma_q'] = 678
catalog.GF_props['omega0_sf'] = 0.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['IGRP'])) == 200, len(np.unique(df['IGRP']))
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129
assert np.isclose(df['WEIGHT'].sum(), 246 * 1.0) # no weights, just 1 per gal
assert np.isclose(df['M_HALO'].sum(), baseline_total_mass)
m2=df['M_HALO'].to_numpy()

catalog = TestGroupCatalog("Test")
#catalog.GF_props['colors'] = 1
catalog.GF_props['omegaL_sf'] = 10.0
catalog.GF_props['sigma_sf'] = 3.0
catalog.GF_props['omegaL_q'] = 0.0
catalog.GF_props['sigma_q'] = 0.0
catalog.GF_props['omega0_sf'] = 10.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['IGRP'])) >= 200 # these parameters make assigned halos smaller
assert len(df) == 246 
assert df['QUIESCENT'].sum() == 129
assert df['WEIGHT'].sum() < 246 
# TODO BUG I feel like this should be true, but it's not. Weighting doesn't preseve the halo mass function
#assert np.isclose(df['M_HALO'].sum(), baseline_total_mass) 
m3=df['M_HALO'].to_numpy()

plt.hist(np.stack([np.log10(m1), np.log10(m2), np.log10(m3)], axis=-1))



print("All tests passed")

In [ ]:
pp.examine_area(np.min(df.RA), np.max(df.RA), np.min(df['DEC']), np.max(df['DEC']), df)

In [ ]:
# Testing repeatability
catalog1 = deepcopy(cat.sdss_colors_chi)
catalog1.caldata = CalibrationData.SDSS_4bin(catalog1.mag_cut, catalog1.GF_props['frac_area'])
catalog1.run_group_finder(popmock=True, silent=True)
previous_hod = catalog1.hod.copy()
gals1 =  main_df = pd.read_csv(catalog1.GF_outfile, delimiter=' ', names=
                          ('RA', 'DEC', 'Z', 'L_GAL', 'VMAX', 'P_SAT', 'M_HALO', 'N_SAT', 'L_TOT', 'IGRP', 'WEIGHT', 'CHI1_WEIGHT'),
                          dtype={'RA': np.float64, 'DEC': np.float64, 'Z': np.float64, 'L_GAL': np.float64, 'VMAX': np.float64,
                                 'P_SAT': np.float64, 'M_HALO': np.float64, 'N_SAT': np.int32, 'L_TOT': np.float64, 'IGRP': np.int64, 'WEIGHT': np.float64, 'CHI1_WEIGHT': np.float64})

catalog2 = deepcopy(cat.sdss_colors_chi)
catalog2.caldata = CalibrationData.SDSS_4bin(catalog2.mag_cut, catalog2.GF_props['frac_area'])
catalog2.run_group_finder(popmock=True, silent=True)
new_hod = catalog2.hod
gals2 =  main_df = pd.read_csv(catalog1.GF_outfile, delimiter=' ', names=
                          ('RA', 'DEC', 'Z', 'L_GAL', 'VMAX', 'P_SAT', 'M_HALO', 'N_SAT', 'L_TOT', 'IGRP', 'WEIGHT', 'CHI1_WEIGHT'),
                          dtype={'RA': np.float64, 'DEC': np.float64, 'Z': np.float64, 'L_GAL': np.float64, 'VMAX': np.float64,
                                 'P_SAT': np.float64, 'M_HALO': np.float64, 'N_SAT': np.int32, 'L_TOT': np.float64, 'IGRP': np.int64, 'WEIGHT': np.float64, 'CHI1_WEIGHT': np.float64})

assert len(gals1) == len(gals2), f"Number of galaxies mismatch: {len(gals1)} vs {len(gals2)}"
assert np.allclose(gals1['RA'], gals2['RA'])
assert np.allclose(gals1['DEC'], gals2['DEC'])
assert np.allclose(gals1['Z'], gals2['Z'])
assert np.allclose(gals1['L_GAL'], gals2['L_GAL'])
assert np.allclose(gals1['VMAX'], gals2['VMAX'])
# TODO BUG This fails when running with parallelization on
#assert np.allclose(gals1['IGRP'], gals2['IGRP']), f"{(gals1['IGRP'] != gals2['IGRP']).sum()} mismatches in IGRP"
#assert np.allclose(gals1['P_SAT'], gals2['P_SAT'])
#assert np.allclose(gals1['M_HALO'], gals2['M_HALO'])
#assert np.allclose(gals1['N_SAT'], gals2['N_SAT'])
#assert np.allclose(gals1['L_TOT'], gals2['L_TOT'])
#assert np.allclose(gals1['WEIGHT'], gals2['WEIGHT'])
assert np.allclose(gals1['CHI1_WEIGHT'], gals2['CHI1_WEIGHT'])


In [ ]:

# Check that the HODs are the same to at least 10% precision. 
# TODO BUG This fails when running with parallelization on. Enough assignments have changed to change HOD
assert np.shape(previous_hod) == np.shape(new_hod), f"Shape mismatch: {np.shape(previous_hod)} vs {np.shape(new_hod)}"
results = np.isclose(previous_hod, new_hod, rtol=0.1, atol=0.0001)
#with np.printoptions(suppress=True, threshold=sys.maxsize, linewidth=500):
    #print(previous_hod)
    #print(results.astype(int).T)
for row in range(len(previous_hod)):
    for col in range(len(previous_hod[row])):
        assert np.isclose(previous_hod[row][col], new_hod[row][col], rtol=0.1, atol=0.0001), f"Row {row}, Col {col} mismatch: {previous_hod[row][col]} vs {new_hod[row][col]}"

In [ ]:
hods_from_cpp = hod.HODTabulated.from_cpp(catalog1.hod, catalog1.caldata) # TODO if this is all set, can save it off (and maybe replace the .hod property)
fig=pp.hod_bins_plot(catalog1, hods_from_cpp)

In [ ]:
# Check that chi squared values are the same
catalog1.calc_wp_for_mock()
result1 = catalog1.chisqr()
catalog2.calc_wp_for_mock()
result2 = catalog2.chisqr()
assert np.isclose(result1, result2), f"Chi squared mismatch: {result1} vs {result2}"

#assert np.isclose(result[0], 94.0, rtol=0.0, atol=3.0), "Chi squared test failed" # pinning previous result

# TODO BUG This fails when running with parallelization on
# This is the results you get from serial execution. Parallel version is quite different!! 
assert np.isclose(result1[0], 107.3, rtol=0.01, atol=1.0), "Chi squared test failed" # pinning previous result

In [ ]:
# Random seeds should all be fixed such that the chi squared is stable from run to run including python preprocessing
catalog = deepcopy(cat.bgs_sv3_pz_2_6_10p_c2)

catalog.preprocess()
catalog.run_group_finder(popmock=True, silent=True)
catalog.calc_wp_for_mock()
result1 = catalog.chisqr()

catalog.preprocess()
catalog.run_group_finder(popmock=True, silent=True)
catalog.calc_wp_for_mock()
result2 = catalog.chisqr()

assert np.isclose(result1[0], result2[0]), "Chi squared should be stable from run to run"
assert np.isclose(result1[0], 690.8, rtol=0.0, atol=0.1), "Chi squared should be stable from run to run"

In [ ]:
catalog = deepcopy(cat.bgs_sv3_pz_2_6_10p_c2)

catalog.preprocess()
catalog.run_group_finder(popmock=True, silent=True)
catalog.calc_wp_for_mock()
result1 = catalog.chisqr()

hods_from_cpp = hod.HODTabulated.from_cpp(catalog.hod, catalog.caldata) # TODO if this is all set, can save it off (and maybe replace the .hod property)
fig=pp.hod_bins_plot(catalog, hods_from_cpp)